In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [7]:
# random.sample(range(0, len(image_names)), 5)

In [8]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 1000)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

1000 municipalities.


In [9]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [11]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [12]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [13]:
# for i,o in val_dl:
#     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1919.014437784354
  Validation Loss:  1731.9552333068848
Loss thresholds for training:  [0.0, 173.19552333068847, 346.39104666137695, 519.5865699920654, 692.7820933227539, 865.9776166534424, 1039.1731399841308, 1212.3686633148193, 1385.5641866455078, 1558.7597099761963]
Starting from threshold:  9  with value:  1558.7597099761963


Epoch:  1
  Training Loss:  1854.3831838480633
  Validation Loss:  1682.7122100067138


Epoch:  2
  Training Loss:  1841.7703684488931
  Validation Loss:  1705.3443585205077


Epoch:  3
  Training Loss:  1827.2689370218914
  Validation Loss:  1690.6817885589599


Epoch:  4
  Training Loss:  1813.637282816569
  Validation Loss:  1702.5269521331786


Epoch:  5
  Training Loss:  1804.5666834513347
  Validation Loss:  1725.5158837127685


Epoch:  6
  Training Loss:  1801.934120534261
  Validation Loss:  1710.6298558044434


